In [2]:
import datetime  
def parseTime(timestamp):	
	local_dt = datetime.datetime.fromtimestamp(timestamp)  
	utc_offset = datetime.timedelta()  
	utc_dt = local_dt + utc_offset  
	
	return utc_dt


def parseAmeTime(timestamp):	
	local_dt = datetime.datetime.fromtimestamp(timestamp)  
	utc_offset = datetime.timedelta(hours=-15)  
	utc_dt = local_dt + utc_offset  
	
	return utc_dt

def truncate_and_format(num, width=10, base=10):  
    # 截断整数（如果需要的话）  
    truncated_num = str(num)[:width] if len(str(num)) > width else str(num)  # 减去冒号和可能的负号  
    # 确保输出至少有指定的宽度
    # formatted_num = truncated_num.zfill(width) if num >= 0 else   truncated_num.zfill(width) 
    formatted_num = truncated_num 
    return formatted_num  

In [ ]:
import urllib.parse  
import hashlib
import time
 
def w_rid(param):
    e = "7cd084941338484aae1ad9425b84077c4932caff0ff746eab6f01bf08b70ac45"
    c = ''.join([e[i] for i in [46, 47, 18, 2, 53, 8, 23, 32, 15, 50, 10, 31, 58, 3, 45, 35, 27, 43, 5, 49, 33, 9, 42, 19, 29, 28, 14, 39, 12, 38, 41, 13, 37, 48, 7, 16, 24, 55, 40, 61, 26, 17, 0, 1, 60, 51, 30, 4, 22, 25, 54, 21, 56, 59, 6, 63, 57, 62, 11, 36, 20, 34, 44, 52]])
    c = c[:32]
    u = int(time.time())    
    param["wts"] = u
    f = [f"{urllib.parse.quote(str(k).encode('utf-8'))}={urllib.parse.quote(str(v).encode('utf-8'))}" for k,v in sorted(param.items())]
    y = '&'.join(f)
    
    return hashlib.md5((y+c).encode(encoding='utf-8')).hexdigest(), u
 
 
print(w_rid({
    "mid": 1111111111111,
    "token": "",
    "platform": "web",
    "web_location": 1550101,
    "dm_img_list": "[]",
    "dm_img_str": "V2ViR0wgMS4wIChPcGVuR0wgRVMgMi4wIENocm9taXVtKQ",
    "dm_cover_img_str": "QU5HTEUgKEludGVsLCBJbnRlbChSKSBVSEQgR3JhcGhpY3MgNjMwICgweDAwMDAzRTlCKSBEaXJlY3QzRDExIHZzXzVfMCBwc181XzAsIEQzRDExKUdvb2dsZSBJbmMuIChJbnRlbC",
    "dm_img_inter": "{\"ds\":[],\"wh\":[3256,5442,64],\"of\":[495,990,495]}"
}
))

('27c2ef136bd3a6c2a481722ce22afe18', 1719895731)


In [12]:
import time
import random
import json

import requests

Cookie="....cookie....."

web_location = 1550101
dm_img_str = "V2ViR0wgMS4wIChPcGVuR0wgRVMgMi4wIENocm9taXVtKQ"
dm_cover_img_str = "QU5HTEUgKEludGVsLCBJbnRlbChSKSBVSEQgR3JhcGhpY3MgNjMwICgweDAwMDAzRTlCKSBEaXJlY3QzRDExIHZzXzVfMCBwc181XzAsIEQzRDExKUdvb2dsZSBJbmMuIChJbnRlbC"
dm_img_inter = "{\"ds\":[],\"wh\":[3256,5442,64],\"of\":[495,990,495]}"

session = requests.Session()  
headers = {  
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
           'Cookie': Cookie 
           }

# 穿靴子的饭团  粉丝  
for pn in range(2,3,1):
    r =random.random()
    time.sleep(r)
    
    limit = 50   # 最大50
    fname = "new"	# 存储文件名
    
    mid = 1111111111111111
    wrid, wts = w_rid({
        "mid": mid,
        "token": "",
        "platform": "web",
        "web_location": web_location,
        "dm_img_list": "[]",
        "dm_img_str": dm_img_str,
        "dm_cover_img_str": dm_cover_img_str,
        "dm_img_inter": dm_img_inter
    })
    url = f"https://api.bilibili.com/x/relation/fans?vmid={mid}&pn={pn}&ps={limit}&order=desc&gaia_source=main_web&web_location={web_location}&w_rid={wrid}&wts={wts}"
    response = session.get(url, headers=headers)  
        
    # 检查响应状态码  
    if response.status_code == 200:  
        # print(response.text)  # 打印返回的内容 
        with open(f"D:/code/bilibili/data-fans/{fname}.json", "w") as fout:
            fout.write(response.text)
            
        fans = json.load(open(f"D:/code/bilibili/data-fans/{fname}.json"))
        for fan in fans['data']['list']:
            mid = fan['mid']
            sign_str = fan['sign'].replace("\n", " ")
            print(f"uid:{truncate_and_format(fan['mid'])}\t最新关注于:{parseTime(fan['mtime'])} AmeTime:{parseAmeTime(fan['mtime'])} name:{fan['uname']:}")
           
            
            wrid, wts = w_rid({
                "mid": mid,
                "token": "",
                "platform": "web",
                "web_location": web_location,
                "dm_img_list": "[]",
                "dm_img_str": dm_img_str,
                "dm_cover_img_str": dm_cover_img_str,
                "dm_img_inter": dm_img_inter
            })
            
            
            url = f"https://space.bilibili.com/{mid}"
            headers["Accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"
            r =random.random()
            time.sleep(r*2)
            response = session.get(url, headers=headers)            
            print(response.status_code, url, response.text[:20])
            
            
            url = f"https://api.bilibili.com/x/space/wbi/acc/info?mid={mid}&token=&platform=web&web_location={web_location}&dm_img_list=[]&dm_img_str={dm_img_str}&dm_cover_img_str={dm_cover_img_str}&dm_img_inter={dm_img_inter}&w_rid={wrid}&wts={wts}"
            headers["Accept"] = "*/*"
            headers["Origin"] = "https://space.bilibili.com"
            headers["Referer"] = f"https://space.bilibili.com/{mid}"
            r =random.random()
            time.sleep(r*2)
            response = session.get(url,headers=headers)
            print(response.status_code, url, end="\n\n")
            if response.status_code == 200:  
                print(response.text)
                with open(f"D:/code/bilibili/profile/{mid}.json", "w") as fout:
                    fout.write(response.text)
                
                profile = json.load(open(f"D:/code/bilibili/profile/{mid}.json"))
                
                sign_str = profile['data']['sign'].replace("\n"," ")  # 签名
                face_str = profile['data']['face'].rsplit("/",1)[1]   # 头像
                school_str = "" if profile['data']['school'] == None else profile['data']['school']['name']  	  # 学校
                print(f"name:{profile['data']['name']} sex:{profile['data']['sex']} birthday:{profile['data']['birthday']} \nface:{face_str}\nsign:{sign_str}\nschool:{school_str}\nbilibili level:{profile['data']['level']}  coins:{profile['data']['coins']}")
                  
    else:  
        print(f"Error: {response.status_code}")
        
        
session.close()

uid:172503365	最新关注于:2024-07-02 12:40:56 AmeTime:2024-07-01 21:40:56 name:zcxka123
200 https://space.bilibili.com/172503365 <!DOCTYPE html><html
200 https://api.bilibili.com/x/space/wbi/acc/info?mid=172503365&token=&platform=web&web_location=1550101&dm_img_list=[]&dm_img_str=V2ViR0wgMS4wIChPcGVuR0wgRVMgMi4wIENocm9taXVtKQ&dm_cover_img_str=QU5HTEUgKEludGVsLCBJbnRlbChSKSBVSEQgR3JhcGhpY3MgNjMwICgweDAwMDAzRTlCKSBEaXJlY3QzRDExIHZzXzVfMCBwc181XzAsIEQzRDExKUdvb2dsZSBJbmMuIChJbnRlbC&dm_img_inter={"ds":[],"wh":[3256,5442,64],"of":[495,990,495]}&w_rid=2cb0cdf35909e9a54ed648fce216b0aa&wts=1719896262

{"code":0,"message":"0","ttl":1,"data":{"mid":172503365,"name":"zcxka123","sex":"保密","face":"https://i0.hdslb.com/bfs/face/member/noface.jpg","face_nft":0,"face_nft_type":0,"sign":"","rank":10000,"level":4,"jointime":0,"moral":0,"silence":0,"coins":0,"fans_badge":false,"fans_medal":{"show":false,"wear":false,"medal":null},"official":{"role":0,"title":"","desc":"","type":-1},"vip":{"type":1,"status":0,"